# Реализация модели защищенного канала передачи данных по спецификации TLS 1.3

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

----

In [43]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410

# Криптография из западных стандартов
from Crypto.Cipher import AES

#
from Crypto.Random import get_random_bytes

#
from functools import partial
import uuid

In [26]:
class KuznyechikMGM(object):
    def __init__(self, key: bytes):
        self._key = key
    
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
        cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, b'')
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        try:
            encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
            cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
            
            ct_with_tag = ciphertext + tag
            
            return cipher.open(nonce, ct_with_tag, b'')
        except (ValueError, KeyError):
            trace('[KuznyechikMGM.decrypt]', f'Incorrect decryption')
            return None
        
    @staticmethod
    def _block_size() -> int:
        return gost3412.GOST3412Kuznechik.blocksize
        
    @staticmethod
    def _encrypt_block(key: bytes, block: bytes):
        cipher = gost3412.GOST3412Kuznechik(key)
        return cipher.encrypt(block)
    
    @staticmethod
    def _generate_nonce() -> bytes:
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

In [27]:
class AESGCM(object):
    def __init__(self, key: bytes):
        self._key = key
        
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        cipher = AES.new(self._key, AES.MODE_GCM)
        return cipher.nonce, *cipher.encrypt_and_digest(plaintext)
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        try:
            cipher = AES.new(self._key, AES.MODE_GCM, nonce=nonce)
            return cipher.decrypt_and_verify(ciphertext, tag)
        except (ValueError, KeyError):
            trace('[AESGCM.decrypt]', f'Incorrect decryption')
            return None

In [28]:
ciphers = {
    'AES_GCM': AESGCM,
    'Kuznyechik_MGM': KuznyechikMGM
}

---

In [30]:
def test_cipher(cipher_id):
    print(f'Testing cipher {cipher_id}...')
    
    cipher_type = ciphers[cipher_id]
    
    key = get_random_bytes(32)
    cipher = cipher_type(key)
    
    pt = get_random_bytes(16)
    success = pt == cipher.decrypt(*cipher.encrypt(pt))
    
    pt = get_random_bytes(17)
    success = success and pt == cipher.decrypt(*cipher.encrypt(pt))
    
    pt = get_random_bytes(32)
    success = success and pt == cipher.decrypt(*cipher.encrypt(pt))
    
    print(f'Result: {success}')
    return success

In [31]:
assert test_cipher('Kuznyechik_MGM')
assert test_cipher('AES_GCM')

Testing cipher Kuznyechik_MGM...
Result: True
Testing cipher AES_GCM...
Result: True


----

In [39]:
class TLS13CertificationAuthority(object):
    def __init__(self, ciphers):
        self._ciphers = ciphers
        
        self._curve = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        self._private_key = gost3410.prv_unmarshal(get_random_bytes(64))
        self._public_key = gost3410.public_key(self._curve, self._private_key)
        
        self._certificates = {}
        self._revoked_certificates = {}
        
        trace('[TLS13CertificationAuthority]', f'Supported ciphers: {list(self._ciphers.keys())}')
        trace('[TLS13CertificationAuthority]', f'Public key = {self._public_key}')
        
    def make_certificate(identifier, public_key):
        

In [50]:
class TLS13Server(object):
    def __init__(self):
        self._id = uuid.uuid4()
        
        trace('[TLS13Server]', f'Id = {self._id}')
        
    @property
    def identifier(self):
        return self._id

In [51]:
class TLS13Client(object):
    def __init__(self, cipher_id):
        self._cipher_type = ciphers[cipher_id]
        self._id = uuid.uuid4()
        
        trace('[TLS13Client]', f'Id = {self._id}, cipher = {cipher_id}')
        
    @property
    def identifier(self):
        return self._id

In [52]:
ca = TLS13CertificationAuthority(ciphers)

[TRACE] [TLS13CertificationAuthority] Supported ciphers: ['AES_GCM', 'Kuznyechik_MGM']
[TRACE] [TLS13CertificationAuthority] Public key = (2157895737115530647994393154282443938718075659904752676596983653235730418190047790910918349525625120946114327037965364296867424712956386937685427549085260, 12540639099394132098205307847575372110364742304046795239608811945288068792753404265038938843534188614376635537006731141555303802806343396889200054649625224)


In [53]:
srv = TLS13Server()

[TRACE] [TLS13Server] Id = 645420ba-d8b8-4c72-a5fe-c362d98d0b5b


In [55]:
cl1 = TLS13Client('AES_GCM')
cl2 = TLS13Client('Kuznyechik_MGM')

[TRACE] [TLS13Client] Id = 237a9dd5-a6f0-4a63-83c8-4ff1ad94da1d, cipher = AES_GCM
[TRACE] [TLS13Client] Id = 6dd79581-ec13-45a0-9c36-ac82f32d54fe, cipher = Kuznyechik_MGM
